In [0]:
%sql 

select cast(created_timestamp as timestamp) as created_timestamp, customer_id, customer_name, cast(date_of_birth as timestamp) as date_of_birth, email, member_since, telephone, file_path from gizmo_box.bronze.v_customers
where customer_id is not null
group by created_timestamp, customer_id, customer_name, date_of_birth, email, member_since, telephone, file_path

In [0]:
%sql
-- use min max grouping

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW v_customers_distinct AS
select cast(created_timestamp as timestamp) as created_timestamp, customer_id, customer_name, cast(date_of_birth as date) as date_of_birth, email, member_since, telephone, file_path from gizmo_box.bronze.v_customers
where customer_id is not null
group by created_timestamp, customer_id, customer_name, date_of_birth, email, member_since, telephone, file_path

In [0]:
%sql
select customer_id, max(created_timestamp) as max_created_timestamp from v_customers_distinct
group by customer_id

In [0]:
%sql
with cte_max as (
    select customer_id, max(created_timestamp) as max_created_timestamp from v_customers_distinct
group by customer_id
)
select t.* from v_customers_distinct t
join cte_max m ON m.customer_id = t.customer_id and m.max_created_timestamp = t.created_timestamp;

In [0]:
%sql 
-- cast to correct datatype (already done)

with cte_max as (
    select customer_id, max(created_timestamp) as max_created_timestamp from v_customers_distinct
group by customer_id
)
select t.* from v_customers_distinct t
join cte_max m ON m.customer_id = t.customer_id and m.max_created_timestamp = t.created_timestamp;

In [0]:
%sql
--write to silver

CREATE TABLE gizmo_box.silver.customers as 
with cte_max as (
    select customer_id, max(created_timestamp) as max_created_timestamp from v_customers_distinct
group by customer_id
)
select t.* from v_customers_distinct t
join cte_max m ON m.customer_id = t.customer_id and m.max_created_timestamp = t.created_timestamp;

In [0]:
%sql
select * from gizmo_box.silver.customers;

In [0]:
%sql
DESCRIBE EXTENDED gizmo_box.silver.customers;